In [1]:
# ================================
# Point-Zero Rebuild: Data Loading & Clean Feature Matrix (Week 2 reset, improved)
# ================================
import pandas as pd, numpy as np, re
from collections import Counter

# ---- Helper: small pretty list
def headlist(x, k=12): 
    return list(x)[:k] + (["..."] if len(x) > k else [])

# ---- 0) Load & normalize column names
Merged = pd.read_csv("merged_full.csv", low_memory=False)
Merged.columns = [str(c).strip().lower() for c in Merged.columns]

print(f"[load] rows={len(Merged):,}, cols={len(Merged.columns)}")
print("[load] sample columns:", headlist(Merged.columns))

# ---- 1) Restrict to endline/follow-up if present
endline_filters = [
    ("followup", lambda s: s == 1),
    ("endline",  lambda s: s == 1),
    ("post",     lambda s: s == 1),
    ("wave",     lambda s: s.astype(str).str.lower().isin(["1","end","endline","post"])),
]
AppliedFilter = None
for col, cond in endline_filters:
    if col in Merged.columns:
        try:
            mask = cond(Merged[col])
            if mask.sum() > 0 and mask.sum() < len(Merged):
                Merged = Merged.loc[mask].copy()
                AppliedFilter = col
                print(f"[restrict] kept endline via '{col}': n={len(Merged):,}")
                break
        except Exception:
            pass
if AppliedFilter is None:
    print("[restrict] no explicit endline/follow-up flag found or usable; using full sample.")

# ---- 2) Choose outcome Y and baseline outcome Y0 (robust patterns)
def pick_first_exist(candidates, where, label):
    for c in candidates:
        if c in where:
            print(f"[choose] {label}: '{c}'")
            return c
    raise KeyError(f"No candidate found for {label}. Tried: {candidates}")

Y_candidates  = ["totalc", "consumption1", "totalc_end", "totalc1", "cons_total1", "consumption_end"]
Y0_candidates = ["totalc0", "consumption0", "totalc_base", "cons_total0", "consumption_baseline"]

Y_col  = pick_first_exist(Y_candidates,  Merged.columns, "endline outcome Y")
Y0_col = pick_first_exist(Y0_candidates, Merged.columns, "baseline outcome Y0")

Y_series  = pd.to_numeric(Merged[Y_col],  errors="coerce")
baselineY = pd.to_numeric(Merged[Y0_col], errors="coerce")

# ---- Optional: log-transform outcome and its baseline counterpart
USE_LOG = True  # flip to False if you want to go back to levels
if USE_LOG:
    Y_series  = np.log1p(Y_series.astype(float))
    baselineY = np.log1p(baselineY.astype(float))
    print(f"[transform] Applied log1p to outcome '{Y_col}' and baseline '{Y0_col}'")
else:
    print(f"[transform] Keeping outcome '{Y_col}' and baseline '{Y0_col}' in levels")

# ---- 3) Treatment (village-level assignments with 0/1/2)
treat_candidates = ["treatment.x", "treated", "treatment", "treat", "assign", "assign_treat", "village_treat"]
treat_col = pick_first_exist(treat_candidates, Merged.columns, "treatment indicator")

# [MOD] Robust parse of tri-valued assignment: 0=control, 1=individual, 2=group
treat_raw = pd.to_numeric(Merged[treat_col], errors="coerce")

treat_any   = (treat_raw > 0).astype("Int64")         # pooled ITT
treat_indiv = (treat_raw == 1).astype("Int64")
treat_group = (treat_raw == 2).astype("Int64")

# [MOD] Diagnostics
vc = treat_raw.value_counts(dropna=False).sort_index()
print(f"[treat] column='{treat_col}', counts:\n{vc.to_string()}")
print(f"[treat] share any={float(treat_any.mean(skipna=True)):.3f}, "
      f"indiv={float(treat_indiv.mean(skipna=True)):.3f}, group={float(treat_group.mean(skipna=True)):.3f}")



# ---- 4) Candidate baseline features (improved rules)
def is_baseline_col(c):
    return bool(re.search(r"(?:^.*0$|_0$|base|baseline|^pre_)", c))

cols = set(Merged.columns)
paired_y = {c for c in cols if c.endswith(".y") and c[:-2]+".x" in cols}
baseline_like = [c for c in cols if (is_baseline_col(c) or c.endswith(".x")) and c not in paired_y]

# Always-keep core baseline controls (demographics & geography) even if they don't match baseline regex
ALWAYS_KEEP = [
    "male", "female", "age", "age.x", "age_head", "age_head0",
    "rship", "relation", "relation_to_head", "rship.x",
    "aimag", "aimag.x", "soum", "soum.x", "region", "region0", "province", "province0",
    "urban", "urban0",
    "hhsize", "hhsize0", "children", "children0", "dependents", "dependents0",
    "educ_head", "educ_head0", "married", "married0",
]

# Exclude obvious identifiers regardless (prevent leakage)
EXCLUDE_IDS = [
    "case_id", "hhid", "household_id", "person_id", "cluster_id", "village_id",
    "rescode", "rescode.x", "rescode.y", "ind", "indid", "enumerator", "survey_id"
]

EXCLUDE_EXPLICIT = {Y_col, Y0_col, treat_col, "baselineY"}
baseline_candidates = sorted(set(baseline_like + [c for c in ALWAYS_KEEP if c in cols])
                             - set(EXCLUDE_IDS) - EXCLUDE_EXPLICIT)

print(f"[features] baseline-like by pattern/.x: {len(baseline_like)}; +always-keep; -ids → {len(baseline_candidates)}")
print("[features] sample baseline candidates:", headlist(baseline_candidates, 20))

# ---- 5) Assemble X_full (baseline-only set) + add treatment and baselineY
X_full = Merged[baseline_candidates].copy()

# Identify numeric vs categorical BEFORE coercion (diagnostics)
types_before = {c: str(Merged[c].dtype) for c in X_full.columns}
num_like_cols = [c for c in X_full.columns if pd.api.types.is_numeric_dtype(Merged[c])]
obj_like_cols = [c for c in X_full.columns if not pd.api.types.is_numeric_dtype(Merged[c])]
print(f"[types] numeric-like={len(num_like_cols)}, non-numeric-like={len(obj_like_cols)}")

# Coerce numerics; keep categoricals as strings for one-hot later
for c in num_like_cols:
    X_full[c] = pd.to_numeric(X_full[c], errors="coerce")
for c in obj_like_cols:
    X_full[c] = Merged[c].astype("string")

# [MOD] Explicit treatment controls (numeric) + baseline outcome
X_full["treat_any"]   = treat_any.astype("float")      # cast to float to avoid Int64 NA quirks later
X_full["treat_indiv"] = treat_indiv.astype("float")
X_full["treat_group"] = treat_group.astype("float")
X_full["baselineY"]   = baselineY

# ---- 6) Row filter: require Y, baselineY, and non-missing raw treatment
keep_rows = (
    Y_series.notna() &
    X_full["baselineY"].notna() &
    treat_raw.notna()
)
n_before = len(X_full)
X_full = X_full.loc[keep_rows].copy()
Y_use  = Y_series.loc[keep_rows].copy()
print(f"[rows] kept rows with non-missing Y / baselineY / treatment: {len(X_full):,} (dropped {n_before - len(X_full):,})")

# ---- 7) Drop columns with extreme missingness (>95% NaN)
missing_rate = X_full.isna().mean().sort_values(ascending=False)
too_missing = list(missing_rate[missing_rate > 0.95].index)

# [MOD] Protect treatment and baselineY from being dropped
PROTECT = {"treat_any","treat_indiv","treat_group","baselineY"}
too_missing = [c for c in too_missing if c not in PROTECT]
if too_missing:
    print(f"[drop] >95% missing: dropping {len(too_missing)} cols")
    X_full = X_full.drop(columns=too_missing)
    
    
# ---- 8) Impute numeric mean; categorical mode (unchanged)
num_cols = [c for c in X_full.columns if pd.api.types.is_numeric_dtype(X_full[c])]
cat_cols = [c for c in X_full.columns if c not in num_cols]

if num_cols:
    X_full[num_cols] = X_full[num_cols].apply(lambda col: col.fillna(col.mean()))
if cat_cols:
    for c in cat_cols:
        mode = X_full[c].mode(dropna=True)
        fill = mode.iloc[0] if not mode.empty else "missing"
        X_full[c] = X_full[c].fillna(fill)

# ---- 8.5) Log-transform eligible numeric X-features (safe log1p)
LOG_PROTECT = {"treat_any", "treat_indiv", "treat_group", "baselineY"}  # never log these

def looks_logged(name: str) -> bool:
    return bool(re.search(r"(?:^log[_]?|[_](log|ln)$|[_]log[_]?|[_]ln$|^ln[_]?)", name))

def is_indicator_like(s: pd.Series) -> bool:
    # Heuristic: very few unique values → likely categorical/indicator
    return s.nunique(dropna=True) <= 10 and set(s.dropna().unique()).issubset({0, 1, 2, 3})

def log_candidate(col: str, s: pd.Series) -> bool:
    if col in LOG_PROTECT:
        return False
    if looks_logged(col):
        return False
    if is_indicator_like(s):
        return False
    # need non-negative values for log1p
    if s.min() < 0:
        return False
    # avoid logging variables that are already tiny/scaled (e.g., shares)
    if s.quantile(0.95) <= 1.0:
        return False
    # avoid columns with too few distinct values
    if s.nunique(dropna=True) < 20:
        return False
    return True

log_applied = []
for c in [c for c in X_full.columns if pd.api.types.is_numeric_dtype(X_full[c])]:
    s = X_full[c]
    if log_candidate(c, s):
        X_full[c] = np.log1p(s.astype(float))
        log_applied.append(c)

print(f"[logX] Applied log1p to {len(log_applied)} numeric features.")
if log_applied:
    print("         e.g.,", headlist(sorted(log_applied), 12))

        
        
# ---- 9) One-hot encode categoricals, then drop zero-variance columns
X_cat = pd.get_dummies(X_full[cat_cols], drop_first=True) if cat_cols else pd.DataFrame(index=X_full.index)
X_num = X_full[num_cols] if num_cols else pd.DataFrame(index=X_full.index)

# [MOD] Ensure protected numerics are present
for k in ["treat_any","treat_indiv","treat_group","baselineY"]:
    if k not in X_num.columns and k in X_full.columns:
        X_num[k] = X_full[k]

X_proc = pd.concat([X_num, X_cat], axis=1)

# Zero variance after impute/encode
nuniq = X_proc.nunique(dropna=True)
zerovar = list(nuniq[nuniq <= 1].index)
# [MOD] Don’t drop protected even if degenerate in a subset
zerovar = [c for c in zerovar if c not in PROTECT]
if zerovar:
    print(f"[drop] zero-variance columns: {len(zerovar)}")
    X_proc = X_proc.drop(columns=zerovar)

    
    # ---- 9.5) Standardize (z-score) all numeric features for ML comparability
# Keep a copy of the scaler means/stds if you want to inverse-transform later
num_cols_proc = [c for c in X_proc.columns if pd.api.types.is_numeric_dtype(X_proc[c])]

# Guard: don't standardize degenerate columns (already removed above), just in case
nondeg = [c for c in num_cols_proc if X_proc[c].std(ddof=0) > 0]

X_scaled_num = X_proc[nondeg].apply(lambda col: (col - col.mean()) / col.std(ddof=0))
X_scaled = pd.concat([X_scaled_num, X_proc.drop(columns=nondeg)], axis=1)

print(f"[scale] standardized {len(nondeg)} numeric features (z-score).")


    
# ---- 10) Drop high-cardinality categorical dummies (likely IDs) as a safety net
K = 100
if not X_cat.empty:
    orig_cat = X_cat.columns.to_series().str.extract(r"^(.*?)[_=].*$", expand=False)
    dummy_counts = orig_cat.value_counts()
    id_like_cats = dummy_counts[dummy_counts > K].index.tolist()
    if id_like_cats:
        drop_cols = [c for c in X_cat.columns if any(c.startswith(name + "_") or c.startswith(name + "=") for name in id_like_cats)]
        # [MOD] No protected vars live in X_cat, but guard anyway
        drop_cols = [c for c in drop_cols if c not in PROTECT]
        if drop_cols:
            print(f"[drop] high-cardinality categorical (>{K} dummies): {id_like_cats} → dropping {len(drop_cols)} cols")
            X_proc = X_proc.drop(columns=drop_cols, errors="ignore")

print(f"[shape] final X_proc: n={X_proc.shape[0]:,}, p={X_proc.shape[1]:,}")
if X_proc.shape[1] == 0:
    print("[warn] zero features after cleaning. Diagnostics:")
    print("  - Top missing columns:\n", missing_rate.head(20))
    print("  - Example dtypes (before):", Counter(types_before.values()))
    print("  - Kept cat_cols:", headlist(cat_cols, 20))
    print("  - Kept num_cols:", headlist(num_cols, 20))


# ---- 11) Build final dataset for modeling (use scaled features)
Data_all = pd.concat([Y_use.rename(Y_col), X_scaled], axis=1)
Y = Data_all[Y_col].to_numpy()
X = Data_all.drop(columns=[Y_col])

print("Data shape (n, p):", X.shape[0], X.shape[1])
print("First few columns:", headlist(X.columns, 20))
print("Confirm treatment columns present:",
      all(k in X.columns for k in ["treat_any","treat_indiv","treat_group"]))



[load] rows=2,109, cols=2416
[load] sample columns: ['case_id', 'hhid', 'rescode.x', 'ind', 'aimag.x', 'soum.x', 'treatment.x', 'followup', 'xxxxxbasichhcharacteristicsxxxxx', 'male', 'rship', 'age.x', '...']
[restrict] kept endline via 'followup': n=961
[choose] endline outcome Y: 'totalc'
[choose] baseline outcome Y0: 'totalc0'
[transform] Applied log1p to outcome 'totalc' and baseline 'totalc0'
[choose] treatment indicator: 'treatment.x'
[treat] column='treatment.x', counts:
treatment.x
0    260
1    350
2    351
[treat] share any=0.729, indiv=0.364, group=0.365
[features] baseline-like by pattern/.x: 896; +always-keep; -ids → 896
[features] sample baseline candidates: ['age.x', 'aimag.x', 'alcohol1.x', 'alcohol10', 'alcohol2.x', 'alcohol20', 'alcohol2_m0', 'alcohol3.x', 'alcohol30', 'alcohol3_m0', 'alcohol4.x', 'alcohol40', 'alcohol4_m0', 'alcohol5.x', 'alcohol50', 'alcohol6.x', 'alcohol60', 'alcohol_c0', 'alcohol_ck10', 'alcohol_ck20', '...']
[types] numeric-like=877, non-numeric-

/tmp/ipykernel_1218/1568368608.py:132: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_full["treat_any"]   = treat_any.astype("float")      # cast to float to avoid Int64 NA quirks later
/tmp/ipykernel_1218/1568368608.py:133: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_full["treat_indiv"] = treat_indiv.astype("float")
/tmp/ipykernel_1218/1568368608.py:134: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all col

[rows] kept rows with non-missing Y / baselineY / treatment: 961 (dropped 0)
[drop] >95% missing: dropping 133 cols
[logX] Applied log1p to 308 numeric features.
         e.g., ['age.x', 'alcohol4.x', 'alcohol40', 'alcohol4_m0', 'alcohol_c0', 'alcohol_cm0', 'alcohol_pp0', 'alcohol_ps0', 'animals0', 'assets_all0', 'bod0', 'books12.x', '...']
[drop] zero-variance columns: 51
[scale] standardized 975 numeric features (z-score).
[drop] high-cardinality categorical (>100 dummies): ['inigiv', 'inirec'] → dropping 256 cols
[shape] final X_proc: n=961, p=719
Data shape (n, p): 961 975
First few columns: ['age.x', 'aimag.x', 'alcohol1.x', 'alcohol10', 'alcohol2.x', 'alcohol20', 'alcohol2_m0', 'alcohol3.x', 'alcohol30', 'alcohol3_m0', 'alcohol4.x', 'alcohol40', 'alcohol4_m0', 'alcohol5.x', 'alcohol50', 'alcohol_c0', 'alcohol_ck10', 'alcohol_ck20', 'alcohol_ckm0', 'alcohol_cm0', '...']
Confirm treatment columns present: True


In [ ]:
# ================================
# Summary Statistics + Visuals (Project One)
# ================================
# Requires: merged_full.csv in the CWD
# Outputs:
#   - table1_baseline.csv / table1_baseline.tex
#   - figs/*.png (Histogram, box/violin, scatter+smooth, aimag bars, appendix densities)
# Notes:
#   - Uses only pandas/numpy/matplotlib; statsmodels LOWESS is optional (guarded).
#   - Does NOT change your core dataset; transforms are temp copies.
# ================================

import os, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---------- Config ----------
INPUT_CSV = "merged_full.csv"
OUT_DIR   = "figs"
os.makedirs(OUT_DIR, exist_ok=True)

# Columns (align with your pipeline / variable naming)
# Outcome + treatment
Y_end_col   = "totalc"            # endline total consumption (will use log)
Y_base_col  = "baselineY"         # your logged baseline consumption aggregate (already log1p in pipeline)
TREAT_GROUP = "treat_group"       # 1 = joint-liability offer, 0 = control
TREAT_ANY   = "treat_any"

# Curated baseline X’s (raw names in your dataset)
vars_enterprise = ["enterprise.x", "soleent.x", "jointent.x", "hours_ent.x", "hours_wage.x", "profit0"]
vars_assets     = ["assets_all0", "animals0", "sheep0", "goats0", "cattle0", "horses0",
                   "dwellingvalue0", "ownsdwelling.x", "dwellingsize.x", "rooms.x"]
vars_income_exp = ["hhincome0", "totalexp0", "foodc0", "nondurc0", "durc0"]
vars_demo       = ["hhsize", "age.x", "edulow.x", "eduvoc.x", "eduhigh.x", "male"]
vars_geo        = ["aimag.x", "soum.x", "monthsliving0", "yearsliving0", "stillresid.x"]
vars_transfers  = ["rec_y0", "gave_y0", "mobile1.x", "tv1.x"]

CURATED_VARS = (
    [Y_base_col] +
    vars_enterprise +
    vars_assets +
    vars_income_exp +
    vars_demo +
    [v for v in vars_geo if v not in ["aimag.x","soum.x"]] +  # keep numeric tenure/land; aimag/soum used for groups/figs
    vars_transfers
)

# Variables to log1p for distributional stability (won’t overwrite raw)
LOG1P_VARS = ["assets_all0", "dwellingvalue0", "hhincome0", "totalexp0",
              "foodc0", "nondurc0", "durc0", "rec_y0", "gave_y0"]

# Helper labels for LaTeX table (left column). Edit as needed.
LABELS = {
    Y_base_col:          "Baseline log consumption (log)",
    "enterprise.x":      "Any enterprise (0/1)",
    "soleent.x":         "Sole enterprise (0/1)",
    "jointent.x":        "Joint enterprise (0/1)",
    "hours_ent.x":       "Hours in enterprise (baseline)",
    "hours_wage.x":      "Hours in wage work (baseline)",
    "profit0":           "Business profit (baseline)",
    "assets_all0":       "Assets (baseline)",
    "animals0":          "Livestock units (baseline)",
    "sheep0":            "Sheep owned (baseline)",
    "goats0":            "Goats owned (baseline)",
    "cattle0":           "Cattle owned (baseline)",
    "horses0":           "Horses owned (baseline)",
    "dwellingvalue0":    "Dwelling value (baseline)",
    "ownsdwelling.x":    "Owns dwelling (0/1)",
    "dwellingsize.x":    "Dwelling size (m$^2$)",
    "rooms.x":           "Number of rooms",
    "hhincome0":         "Income (baseline)",
    "totalexp0":         "Total expenditure (baseline)",
    "foodc0":            "Food expenditure (baseline)",
    "nondurc0":          "Nondurables (baseline)",
    "durc0":             "Durables (baseline)",
    "hhsize":            "Household size",
    "age.x":             "Avg. age (baseline)",
    "edulow.x":          "Primary or less (0/1)",
    "eduvoc.x":          "Vocational (0/1)",
    "eduhigh.x":         "High school+ (0/1)",
    "male":              "Male head (0/1)",
    "monthsliving0":     "Months in community",
    "yearsliving0":      "Years in community",
    "stillresid.x":      "Still resident (0/1)",
    "rec_y0":            "Transfers received (baseline)",
    "gave_y0":           "Transfers given (baseline)",
    "mobile1.x":         "Owns mobile (0/1)",
    "tv1.x":             "Owns TV (0/1)",
}

# ===== Compatibility shim: align with your cleaning pipeline =====
import pandas as pd, numpy as np, re, os, math
import matplotlib.pyplot as plt

INPUT_CSV = "merged_full.csv"
OUT_DIR   = "figs"
os.makedirs(OUT_DIR, exist_ok=True)

# 0) Load + lowercase columns (matches your pipeline)
df = pd.read_csv(INPUT_CSV, low_memory=False)
df.columns = [str(c).strip().lower() for c in df.columns]

# 1) Identify endline and baseline outcome columns (like your pick_first_exist)
Y_candidates  = ["totalc","consumption1","totalc_end","totalc1","cons_total1","consumption_end"]
Y0_candidates = ["totalc0","consumption0","totalc_base","cons_total0","consumption_baseline"]

def pick_first_exist(cands, cols, label):
    for c in cands:
        if c in cols:
            print(f"[choose] {label}: '{c}'")
            return c
    raise KeyError(f"No candidate found for {label}. Tried: {cands}")

Y_col  = pick_first_exist(Y_candidates,  df.columns, "endline outcome Y")
Y0_col = pick_first_exist(Y0_candidates, df.columns, "baseline outcome Y0")

# Create log1p baselineY (to mirror your pipeline)
df["baseliney"] = np.log1p(pd.to_numeric(df[Y0_col], errors="coerce"))
# Also keep a logged endline outcome for figures if needed later
df["log_" + Y_col] = np.log1p(pd.to_numeric(df[Y_col], errors="coerce"))

# 2) Recover treatment from tri-valued assignment (0=control, 1=individual, 2=group)
treat_candidates = ["treatment.x","treated","treatment","treat","assign","assign_treat","village_treat"]
treat_col = pick_first_exist(treat_candidates, df.columns, "treatment indicator")

treat_raw = pd.to_numeric(df[treat_col], errors="coerce")
df["treat_any"]   = (treat_raw > 0).astype("Int64")
df["treat_indiv"] = (treat_raw == 1).astype("Int64")
df["treat_group"] = (treat_raw == 2).astype("Int64")

print(f"[treat] column='{treat_col}', shares any={float(df['treat_any'].mean(skipna=True)):.3f}, "
      f"indiv={float(df['treat_indiv'].mean(skipna=True)):.3f}, group={float(df['treat_group'].mean(skipna=True)):.3f}")


# ---------- Balance / Summary Table ----------
def std_diff(x_t, x_c):
    """Standardized difference: (mean_t - mean_c) / pooled sd."""
    mt, mc = np.nanmean(x_t), np.nanmean(x_c)
    vt, vc = np.nanvar(x_t, ddof=1), np.nanvar(x_c, ddof=1)
    # Pooled SD (two-sample, unweighted by group sizes in denominator as in std-diff convention)
    sp = math.sqrt((vt + vc) / 2.0) if np.isfinite(vt) and np.isfinite(vc) else np.nan
    return (mt - mc) / sp if sp and sp > 0 else np.nan

def summarize_table(df, vars_list, treat_col=TREAT_GROUP, treat_val=1, dropna_axis=0):
    # Filter groups
    treated = df.loc[df[treat_col] == treat_val]
    control = df.loc[df[treat_col] == 0]

    rows = []
    for v in vars_list:
        if v not in df.columns:
            continue
        x_all = df[v]
        x_t   = treated[v]
        x_c   = control[v]

        mean_all = np.nanmean(x_all)
        sd_all   = np.nanstd(x_all, ddof=1)
        mean_c   = np.nanmean(x_c)
        mean_t   = np.nanmean(x_t)
        sdiff    = std_diff(x_t, x_c)

        rows.append({
            "variable": v,
            "label": LABELS.get(v, v),
            "Mean (All)": mean_all,
            "SD (All)": sd_all,
            "Mean (Control)": mean_c,
            "Mean (JL Offer)": mean_t,
            "Std. Diff": sdiff,
            "N (All)": x_all.notna().sum()
        })

    tab = pd.DataFrame(rows)
    # Order: put baselineY first, then logical groupings
    return tab

# Build the final list (keep existing columns only)
final_vars = [v for v in CURATED_VARS if v in df.columns]


# --- after loading CSV and lowercasing columns ---

# 1) Fix baselineY naming and config
Y_base_col = "baseliney"  # keep config in sync with lowercase creation
df["baseliney"] = np.log1p(pd.to_numeric(df[Y0_col], errors="coerce"))

# 2) Create log1p columns you reference later (assets, etc.)
for c in ["assets_all0","dwellingvalue0","hhincome0","totalexp0","foodc0","nondurc0","durc0","rec_y0","gave_y0"]:
    if c in df.columns:
        df[f"log1p_{c}"] = np.log1p(pd.to_numeric(df[c], errors="coerce"))

# 3) Explicit recodes for supposed 0/1 indicators (defensive)
#    Adjust these mappings to your actual codebook if needed.
def binarize(series, true_values={1, "1", "yes", "y", True}):
    x = pd.Series(series).copy()
    # try numeric first
    xn = pd.to_numeric(x, errors="coerce")
    # if coded as 1/2/3, treat 1 as owner/yes, else 0
    out = np.where(xn == 1, 1, np.where(xn == 0, 0, np.nan))
    # fallback: strings
    mask = pd.isna(out)
    out[mask] = np.where(x[mask].astype(str).str.strip().str.lower().isin({str(v).lower() for v in true_values}), 1, 0)
    return pd.to_numeric(out, errors="coerce")

binary_vars = ["enterprise.x","soleent.x","jointent.x","ownsdwelling.x","edulow.x","eduvoc.x","eduhigh.x","male","stillresid.x","mobile1.x","tv1.x"]
for v in binary_vars:
    if v in df.columns:
        # only recode if values look non-binary
        vals = pd.to_numeric(df[v], errors="coerce")
        if not set(pd.unique(vals.dropna())).issubset({0,1}):
            df[v] = binarize(df[v])

# 4) Baseline/endline enterprise columns: ensure you have both and label correctly
#    If your dataset uses enterprise.y for endline, keep baseline in .x and endline in .y
if "enterprise.y" in df.columns and "enterprise.x" in df.columns:
    df["enterprise_baseline"] = binarize(df["enterprise.x"])
    df["enterprise_endline"]  = binarize(df["enterprise.y"])
    # (Optionally) update your CURATED_VARS and LABELS to use these canonical names
    LABELS["enterprise_baseline"] = "Any enterprise at baseline (0/1)"
    LABELS["enterprise_endline"]  = "Any enterprise at endline (0/1)"


table1 = summarize_table(df, final_vars, treat_col=TREAT_GROUP, treat_val=1)
# Save CSV
table1.to_csv("table1_baseline.csv", index=False)

# Pretty LaTeX (minimal formatting; you can style in Overleaf)
def to_latex_table(tab, fname="table1_baseline.tex", caption="Baseline Characteristics (Curated)", label="tab:baseline"):
    show = tab.copy()
    show = show[["label","Mean (All)","SD (All)","Mean (Control)","Mean (JL Offer)","Std. Diff","N (All)"]]
    with open(fname, "w") as f:
        f.write("\\begin{table}[H]\\centering\n")
        f.write("\\caption{" + caption + "}\n")
        f.write("\\label{" + label + "}\n")
        f.write("\\begin{tabular}{lrrrrrr}\n\\toprule\n")
        f.write("Variable & Mean (All) & SD (All) & Mean (Control) & Mean (JL Offer) & Std. Diff & N \\\\\n\\midrule\n")
        for _, r in show.iterrows():
            f.write(f"{r['label']} & {r['Mean (All)']:.3f} & {r['SD (All)']:.3f} & {r['Mean (Control)']:.3f} & {r['Mean (JL Offer)']:.3f} & {r['Std. Diff']:.3f} & {int(r['N (All)'])} \\\\\n")
        f.write("\\bottomrule\n\\end{tabular}\n")
        f.write("\\begin{minipage}{0.92\\linewidth}\\vspace{0.6ex}\\footnotesize\n")
        f.write("Notes: Means and SDs computed using household weights if applicable (not applied here). Standardized differences use pooled SD. Variables with heavy right tails appear log-transformed in figures (log1p). Standard errors in later sections are clustered at the village level.\n")
        f.write("\\end{minipage}\n")
        f.write("\\end{table}\n")

to_latex_table(table1)

print("[ok] Wrote table1_baseline.csv and table1_baseline.tex")

# ---------- Figures ----------
# Helper: histogram
def save_hist(col, bins=30, fname="hist.png", title=None, xlabel=None):
    x = df[col].dropna().values
    plt.figure(figsize=(6,4.2))
    plt.hist(x, bins=bins)
    if title: plt.title(title)
    if xlabel: plt.xlabel(xlabel)
    plt.ylabel("Count")
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, fname), dpi=300)
    plt.close()

# Helper: violin/box by group
def save_violin_by(col, group_col, fname="violin.png", title=None, labels=("0","1")):
    g0 = df.loc[df[group_col]==0, col].dropna().values
    g1 = df.loc[df[group_col]==1, col].dropna().values
    plt.figure(figsize=(6,4.2))
    parts = plt.violinplot([g0, g1], showmeans=True, showextrema=False, showmedians=False)
    plt.xticks([1,2], labels)
    if title: plt.title(title)
    plt.ylabel(col)
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, fname), dpi=300)
    plt.close()

def save_box_by(col, group_col, fname="box.png", title=None, labels=("0","1")):
    g0 = df.loc[df[group_col]==0, col].dropna().values
    g1 = df.loc[df[group_col]==1, col].dropna().values
    plt.figure(figsize=(6,4.2))
    plt.boxplot([g0, g1], showmeans=True)
    plt.xticks([1,2], labels)
    if title: plt.title(title)
    plt.ylabel(col)
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, fname), dpi=300)
    plt.close()

# Helper: scatter + simple smooth (poly or LOWESS if statsmodels available)
def save_scatter_smooth(xcol, ycol, fname="scatter.png", title=None, xl=None, yl=None):
    x = df[[xcol, ycol]].dropna()
    X = x[xcol].values
    Y = x[ycol].values

    plt.figure(figsize=(6,4.2))
    plt.scatter(X, Y, s=8, alpha=0.5)
    # LOWESS if available
    try:
        from statsmodels.nonparametric.smoothers_lowess import lowess
        z = lowess(Y, X, frac=0.25, it=0, return_sorted=True)
        plt.plot(z[:,0], z[:,1])
    except Exception:
        # Fallback: 3rd-degree polynomial fit
        p = np.poly1d(np.polyfit(X, Y, 3))
        xs = np.linspace(X.min(), X.max(), 200)
        plt.plot(xs, p(xs))

    if title: plt.title(title)
    if xl: plt.xlabel(xl)
    if yl: plt.ylabel(yl)
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, fname), dpi=300)
    plt.close()

# Helper: bar by aimag (mean baseline consumption, enterprise share)
def save_bar_aimag(val_col, fname="bar.png", title=None, ylabel=None):
    if "aimag.x" not in df.columns:
        return
    tmp = df[["aimag.x", val_col]].dropna().groupby("aimag.x")[val_col].mean().sort_values()
    plt.figure(figsize=(7.5,4.2))
    plt.bar(tmp.index.astype(str), tmp.values)
    if title: plt.title(title)
    plt.xlabel("Aimag")
    if ylabel: plt.ylabel(ylabel)
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, fname), dpi=300)
    plt.close()

def save_bar_aimag_share(ind_col, fname="bar_share.png", title=None, ylabel=None):
    if "aimag.x" not in df.columns:
        return
    g = df[["aimag.x", ind_col]].dropna()
    tmp = g.groupby("aimag.x")[ind_col].mean().sort_values()
    plt.figure(figsize=(7.5,4.2))
    plt.bar(tmp.index.astype(str), tmp.values)
    if title: plt.title(title)
    plt.xlabel("Aimag")
    if ylabel: plt.ylabel(ylabel)
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, fname), dpi=300)
    plt.close()

# ---------- Produce figures ----------
# Figure 1: Histograms (baseline log consumption, assets log1p)
if Y_base_col in df.columns:
    save_hist(Y_base_col, bins=30, fname="hist_baselineY.png",
              title="Baseline Consumption (log)", xlabel="log(consumption)")

if "log1p_assets_all0" in df.columns:
    save_hist("log1p_assets_all0", bins=30, fname="hist_assets_log1p.png",
              title="Assets (log1p)", xlabel="log(assets + 1)")

# Figure 2: Enterprise vs non-enterprise distributions
if "enterprise.x" in df.columns and Y_base_col in df.columns:
    # Box/Violin for baseline consumption by enterprise status
    save_violin_by(Y_base_col, "enterprise.x", fname="violin_baselineY_by_enterprise.png",
                   title="Baseline Consumption by Enterprise Status", labels=("No enterprise","Has enterprise"))
    save_box_by(Y_base_col, "enterprise.x", fname="box_baselineY_by_enterprise.png",
                title="Baseline Consumption by Enterprise Status", labels=("No enterprise","Has enterprise"))

if "profit0" in df.columns and "enterprise.x" in df.columns:
    # Profit only among enterprise==1 (to avoid many zeros)
    sub = df.loc[df["enterprise.x"]==1, "profit0"].dropna()
    if len(sub) > 0:
        plt.figure(figsize=(6,4.2))
        plt.hist(sub.values, bins=30)
        plt.title("Baseline Profits (enterprise households)")
        plt.xlabel("profit0")
        plt.ylabel("Count")
        plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, "hist_profit_enterprise.png"), dpi=300)
        plt.close()

# Figure 3: Scatter assets vs baseline log consumption (with smooth)
if "log1p_assets_all0" in df.columns and Y_base_col in df.columns:
    save_scatter_smooth("log1p_assets_all0", Y_base_col,
                        fname="scatter_assets_vs_baselineY.png",
                        title="Assets vs Baseline Consumption",
                        xl="log(assets+1)", yl="log(consumption)")

# Figure 4: Aimag bars (means)
if "aimag.x" in df.columns:
    # Mean baseline consumption by aimag
    if Y_base_col in df.columns:
        save_bar_aimag(Y_base_col, fname="bar_aimag_baselineY.png",
                       title="Baseline Consumption by Aimag (mean, log)",
                       ylabel="log(consumption)")
    # Enterprise share by aimag
    if "enterprise.x" in df.columns:
        save_bar_aimag_share("enterprise.x", fname="bar_aimag_enterprise_share.png",
                             title="Enterprise Share by Aimag", ylabel="Share")

# Appendix densities: hours and transfers (log1p)
def save_density(col, fname):
    x = df[col].dropna().values
    if len(x) == 0: return
    plt.figure(figsize=(6,4.2))
    # simple density via hist with many bins; keep neutral style
    plt.hist(x, bins=40, density=True)
    plt.title(col + " (density)")
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, fname), dpi=300)
    plt.close()

for col in ["hours_ent.x","hours_wage.x","log1p_rec_y0","log1p_gave_y0"]:
    if col in df.columns:
        save_density(col, fname=f"density_{col}.png")

print(f"[ok] Figures saved to ./{OUT_DIR}/")


In [ ]:
# Summary Statistics—Outcomes
# ================================
# Summary stats (mean, SD, N) + histograms for LOGGED outcomes
# ================================
import os, numpy as np, pandas as pd
import matplotlib.pyplot as plt

os.makedirs("tables", exist_ok=True)
os.makedirs("figs", exist_ok=True)

# --- 0) Collect outcome series (log1p scale)
# Primary endline and baseline (already log1p if USE_LOG=True in your cleaner)
outcomes = {
    f"log1p_{Y_col}": Y_use,          # endline outcome on log1p scale
    f"log1p_{Y0_col}": baselineY,     # baseline outcome on log1p scale
}

# (Optional) also show level-scale histograms by back-transforming log1p
MAKE_LEVEL_HISTS = True
GROUP_BY_Z = False   # set True to also make summary by treatment assignment Z (ITT)

# --- 1) Helpers
def freedman_diaconis_bins(x, max_bins=80):
    x = pd.Series(x).dropna().values
    if x.size < 2:
        return 10
    q75, q25 = np.percentile(x, [75, 25])
    iqr = q75 - q25
    if iqr <= 0:
        return min(30, max_bins)
    h = 2 * iqr * (x.size ** (-1/3))
    if h <= 0:
        return min(30, max_bins)
    bins = int(np.ceil((x.max() - x.min()) / h))
    return max(10, min(bins, max_bins))

def to_latex_booktabs(df, caption, label, colformats=None):
    df_fmt = df.copy()
    out = []
    out.append("\\begin{table}[!htbp]\\centering")
    out.append(f"\\caption{{{caption}}}")
    out.append(f"\\label{{{label}}}")
    out.append("\\begin{threeparttable}")
    # column alignment
    cols = "l" + "r" * (df_fmt.shape[1])
    out.append(f"\\begin{{tabular}}{{{cols}}}")
    out.append("\\toprule")
    out.append(" & " + " & ".join(df_fmt.columns) + " \\\\")
    out.append("\\midrule")
    for idx, row in df_fmt.iterrows():
        out.append(str(idx) + " & " + " & ".join(map(str, row.values)) + " \\\\")
    out.append("\\bottomrule")
    out.append("\\begin{tablenotes}[flushleft]")
    out.append("\\footnotesize \\textit{Notes}: Statistics computed on the log1p scale for outcomes. "
               "Geometric mean back-transform for levels is $\\exp(\\mathbb{E}[\\log(1+Y)])-1$, "
               "which is \\emph{not} the arithmetic mean of $Y$.")
    out.append("\\end{tablenotes}")
    out.append("\\end{threeparttable}")
    out.append("\\end{table}")
    return "\n".join(out)

def summarize_series(s: pd.Series, name: str):
    s = pd.to_numeric(s, errors="coerce")
    n = int(s.notna().sum())
    mu = float(s.mean())
    sd = float(s.std(ddof=1))
    # back-transform geometric mean for levels, given log1p
    gmean_level = float(np.expm1(mu))
    return {
        "N": n,
        "Mean (log1p)": f"{mu:.3f}",
        "SD (log1p)": f"{sd:.3f}",
        "Geometric mean (level)": f"{gmean_level:.3f}",
    }

# --- 2) Build main summary table
rows = []
for name, s in outcomes.items():
    rows.append((name, summarize_series(s, name)))
Summary = pd.DataFrame({k: v for k, v in rows}).T[["N","Mean (log1p)","SD (log1p)","Geometric mean (level)"]]

Summary.to_csv("tables/outcome_summary.csv")
with open("tables/outcome_summary.tex","w") as f:
    f.write(to_latex_booktabs(Summary, "Outcome Summary Statistics (Log Scale)", "tab:outcome_summary"))
print("[write] tables/outcome_summary.csv, tables/outcome_summary.tex")

# --- 3) Histograms (log scale, and optional level scale)
for name, s in outcomes.items():
    x = pd.to_numeric(s, errors="coerce").dropna()
    if x.empty:
        print(f"[hist] {name}: no data to plot.")
        continue
    bins = freedman_diaconis_bins(x)
    # LOG histogram
    plt.figure(figsize=(6,4))
    plt.hist(x, bins=bins, edgecolor="black", alpha=0.8)
    plt.axvline(x.mean(), linestyle="--", linewidth=1)
    plt.axvline(np.median(x), linestyle=":", linewidth=1)
    plt.title(f"Histogram — {name}")
    plt.xlabel(f"{name} (log1p scale)")
    plt.ylabel("Count")
    plt.tight_layout()
    fn_log = f"figs/hist_log_{name}.png".replace(" ", "_")
    plt.savefig(fn_log, dpi=300); plt.close()
    print(f"[write] {fn_log}")
    # LEVEL histogram (optional)
    if MAKE_LEVEL_HISTS:
        x_level = np.expm1(x)
        binsL = freedman_diaconis_bins(x_level)
        plt.figure(figsize=(6,4))
        plt.hist(x_level, bins=binsL, edgecolor="black", alpha=0.8)
        plt.axvline(x_level.mean(), linestyle="--", linewidth=1)
        plt.axvline(np.median(x_level), linestyle=":", linewidth=1)
        plt.title(f"Histogram — {name} (levels)")
        plt.xlabel(f"{name.replace('log1p_', '')} (level scale)")
        plt.ylabel("Count")
        plt.tight_layout()
        fn_lv = f"figs/hist_level_{name}.png".replace(" ", "_")
        plt.savefig(fn_lv, dpi=300); plt.close()
        print(f"[write] {fn_lv}")

# --- 4) (Optional) By-assignment summary (ITT-flavored descriptive)
if GROUP_BY_Z and "treat_any" in X_proc.columns:
    # Pull Z for the kept rows
    Z = X_proc.loc[Y_use.index, "treat_any"]
    byZ_rows = []
    for name, s in outcomes.items():
        s_use = pd.to_numeric(s, errors="coerce")
        df_tmp = pd.DataFrame({"y": s_use, "Z": Z}).dropna()
        for z in (0.0, 1.0):
            y_z = df_tmp.loc[df_tmp["Z"]==z, "y"]
            n = int(y_z.notna().sum())
            mu = float(y_z.mean()); sd = float(y_z.std(ddof=1))
            gmean_level = float(np.expm1(mu))
            byZ_rows.append({
                "Outcome": name, "Group": f"Z={int(z)}",
                "N": n, "Mean (log1p)": f"{mu:.3f}",
                "SD (log1p)": f"{sd:.3f}",
                "Geometric mean (level)": f"{gmean_level:.3f}"
            })
    ByZ = (pd.DataFrame(byZ_rows)
             .set_index(["Outcome","Group"])
             [["N","Mean (log1p)","SD (log1p)","Geometric mean (level)"]])
    ByZ.to_csv("tables/outcome_summary_byZ.csv")
    # compact LaTeX
    lines = []
    lines.append("\\begin{table}[!htbp]\\centering")
    lines.append("\\caption{Outcome Summary by Treatment Assignment (Log Scale)}")
    lines.append("\\label{tab:outcome_summary_byZ}")
    lines.append("\\begin{threeparttable}")
    lines.append("\\begin{tabular}{llrrrr}")
    lines.append("\\toprule")
    lines.append("Outcome & Group & N & Mean (log1p) & SD (log1p) & Geometric mean (level)\\\\")
    lines.append("\\midrule")
    for (out, grp), row in ByZ.iterrows():
        lines.append(f"{out} & {grp} & {row['N']} & {row['Mean (log1p)']} & {row['SD (log1p)']} & {row['Geometric mean (level)']}\\\\")
    lines.append("\\bottomrule")
    lines.append("\\begin{tablenotes}[flushleft]")
    lines.append("\\footnotesize \\textit{Notes}: Descriptive only; causal ITT is estimated in regression tables.")
    lines.append("\\end{tablenotes}")
    lines.append("\\end{threeparttable}")
    lines.append("\\end{table}")
    with open("tables/outcome_summary_byZ.tex","w") as f:
        f.write("\n".join(lines))
    print("[write] tables/outcome_summary_byZ.csv, tables/outcome_summary_byZ.tex")


In [ ]:
# --- Diagnostics + explicit output dir for tables ---
import os
TABLE_DIR = "tables"
os.makedirs(TABLE_DIR, exist_ok=True)

print(f"[cwd] {os.getcwd()}")
print(f"[table] rows={len(table1)}; non-null vars={table1['variable'].nunique()}")

# Save CSV + LaTeX into tables/
csv_path = os.path.join(TABLE_DIR, "table1_baseline.csv")
tex_path = os.path.join(TABLE_DIR, "table1_baseline.tex")
table1.to_csv(csv_path, index=False)

def to_latex_table(tab, fname, caption="Baseline Characteristics (Curated)", label="tab:baseline"):
    show = tab[["label","Mean (All)","SD (All)","Mean (Control)","Mean (JL Offer)","Std. Diff","N (All)"]].copy()
    with open(fname, "w") as f:
        f.write("\\begin{table}[H]\\centering\n")
        f.write("\\caption{" + caption + "}\n")
        f.write("\\label{" + label + "}\n")
        f.write("\\begin{tabular}{lrrrrrr}\n\\toprule\n")
        f.write("Variable & Mean (All) & SD (All) & Mean (Control) & Mean (JL Offer) & Std. Diff & N \\\\\n\\midrule\n")
        for _, r in show.iterrows():
            f.write(f"{r['label']} & {r['Mean (All)']:.3f} & {r['SD (All)']:.3f} & "
                    f"{r['Mean (Control)']:.3f} & {r['Mean (JL Offer)']:.3f} & "
                    f"{r['Std. Diff']:.3f} & {int(r['N (All)'])} \\\\\n")
        f.write("\\bottomrule\n\\end{tabular}\n")
        f.write("\\begin{minipage}{0.92\\linewidth}\\vspace{0.6ex}\\footnotesize\n")
        f.write("Notes: Standardized differences use pooled SD. Heavy-tailed variables are log-transformed in figures. ")
        f.write("Village-level clustering is used for inference in subsequent sections.\n")
        f.write("\\end{minipage}\n\\end{table}\n")

to_latex_table(table1, tex_path)
print(f"[ok] wrote {csv_path}")
print(f"[ok] wrote {tex_path}")


In [ ]:
# ===========================================
# Table 2 — Outcomes (baseline & endline) with cluster-robust ITT
# ===========================================
import os, math
import numpy as np
import pandas as pd
import statsmodels.api as sm

# ---------- ensure output dir ----------
TABLE_DIR = "tables"
os.makedirs(TABLE_DIR, exist_ok=True)

# ---------- helpers ----------
def pick_first_exist(cands, cols, label):
    for c in cands:
        if c in cols:
            print(f"[choose] {label}: '{c}'")
            return c
    print(f"[skip] none found for {label} among {cands}")
    return None

def std_diff(x_t, x_c):
    mt, mc = np.nanmean(x_t), np.nanmean(x_c)
    vt, vc = np.nanvar(x_t, ddof=1), np.nanvar(x_c, ddof=1)
    if not np.isfinite(vt) or not np.isfinite(vc):
        return np.nan
    sp = math.sqrt((vt + vc) / 2.0)
    return (mt - mc) / sp if sp > 0 else np.nan

def find_cluster_col(cols):
    for c in ["village_id","village","cluster_id","cluster","psu","psu_id","community","community_id","rescode","rescode.x","rescode.y"]:
        if c in cols:
            print(f"[choose] cluster: '{c}'")
            return c
    print("[warn] no cluster id column found; will use HC2 robust SEs.")
    return None

def itt_cluster_robust(df_run, ycol, tcol, cluster_col=None):
    y = pd.to_numeric(df_run[ycol], errors="coerce")
    d = pd.to_numeric(df_run[tcol], errors="coerce")
    ok = y.notna() & d.notna()
    if ok.sum() == 0 or pd.Series(d[ok]).nunique() < 2:
        return np.nan, np.nan
    X = sm.add_constant(d[ok].to_numpy())
    yv = y[ok].to_numpy()
    model = sm.OLS(yv, X, missing='drop')
    if cluster_col is not None and (cluster_col in df_run.columns):
        g = df_run.loc[ok, cluster_col]
        fit = model.fit(cov_type='cluster', cov_kwds={'groups': g})
    else:
        fit = model.fit(cov_type='HC2')
    return float(fit.params[1]), float(fit.bse[1])

# ---------- label map ----------
LABELS_OUT = {
    "Y0_main": "Baseline consumption (log1p)",
    "Y1_main": "Endline consumption (log1p)",
    "food0":   "Baseline food exp. (log1p)",
    "food1":   "Endline food exp. (log1p)",
    "profit0": "Baseline business profit (levels)",
    "profit1": "Endline business profit (levels)",
    "enterprise0": "Baseline enterprise (0/1)",
    "enterprise1": "Endline enterprise (0/1)",
}

# ---------- build outcomes list from what actually exists ----------
outcomes = []

# Required: main consumption
if "baseliney" in df.columns:
    outcomes.append(("Y0_main", "baseliney", True))
else:
    # build it from baseline column if present
    Y0_candidates = ["totalc0","consumption0","totalc_base","cons_total0","consumption_baseline"]
    Y0_col = pick_first_exist(Y0_candidates, df.columns, "baseline outcome Y0 (for baseliney)")
    if Y0_col:
        df["baseliney"] = np.log1p(pd.to_numeric(df[Y0_col], errors="coerce"))
        outcomes.append(("Y0_main", "baseliney", True))

# Endline main
if "log_" + Y_col in df.columns:
    outcomes.append(("Y1_main", "log_" + Y_col, False))
else:
    df["log_" + Y_col] = np.log1p(pd.to_numeric(df[Y_col], errors="coerce"))
    outcomes.append(("Y1_main", "log_" + Y_col, False))

# Optional: food consumption baseline/endline
food_base = pick_first_exist(["foodc0","food0","food_base"], df.columns, "baseline food")
if food_base:
    df["log1p_"+food_base] = np.log1p(pd.to_numeric(df[food_base], errors="coerce"))
    outcomes.append(("food0", "log1p_"+food_base, True))

food_end = pick_first_exist(["foodc1","foodc_end","food1","cons_food1","cons_food_end"], df.columns, "endline food")
if food_end:
    df["log1p_"+food_end] = np.log1p(pd.to_numeric(df[food_end], errors="coerce"))
    outcomes.append(("food1", "log1p_"+food_end, False))

# Optional: profits baseline/endline (levels)
if "profit0" in df.columns:
    df["profit0"] = pd.to_numeric(df["profit0"], errors="coerce")
    outcomes.append(("profit0","profit0", True))
profit1 = pick_first_exist(["profit1","profit_end","profits1","profits_end"], df.columns, "endline profit")
if profit1:
    df[profit1] = pd.to_numeric(df[profit1], errors="coerce")
    outcomes.append(("profit1", profit1, False))

# Optional: enterprise indicator baseline/endline
ent0 = pick_first_exist(["enterprise0","enterprise_base","enterprise.x"], df.columns, "baseline enterprise")
if ent0:
    outcomes.append(("enterprise0", ent0, True))
ent1 = pick_first_exist(["enterprise1","enterprise_end","enterprise.y"], df.columns, "endline enterprise")
if ent1:
    outcomes.append(("enterprise1", ent1, False))

# If nothing made it in:
if len(outcomes) == 0:
    raise RuntimeError("[abort] No outcomes detected. Check column names and re-run.")

# ---------- cluster col ----------
cluster_col = find_cluster_col(df.columns)

# ---------- compute table ----------
rows = []
missing_cols = []
for code, col, is_base in outcomes:
    if col not in df.columns:
        missing_cols.append(col)
        continue

    sub = df[[col, "treat_group"]].copy()
    sub["treat_group"] = pd.to_numeric(sub["treat_group"], errors="coerce")
    sub = sub.dropna(subset=[col, "treat_group"])
    if sub.empty:
        print(f"[skip] outcome '{col}' has no usable data after dropping NA.")
        continue

    all_mean = sub[col].mean()
    all_sd   = sub[col].std(ddof=1)
    m_c      = sub.loc[sub["treat_group"]==0, col].mean()
    m_t      = sub.loc[sub["treat_group"]==1, col].mean()

    sdiff = np.nan
    if is_base:
        sdiff = std_diff(sub.loc[sub["treat_group"]==1, col],
                         sub.loc[sub["treat_group"]==0, col])

    # ITT with robust SE
    if cluster_col:
        itt, se = itt_cluster_robust(df[[col,"treat_group",cluster_col]], col, "treat_group", cluster_col)
    else:
        itt, se = itt_cluster_robust(df[[col,"treat_group"]], col, "treat_group", None)

    rows.append({
        "Outcome": LABELS_OUT.get(code, code),
        "N": int(sub.shape[0]),
        "Mean (All)": all_mean,
        "SD (All)": all_sd,
        "Mean (Control)": m_c,
        "Mean (JL Offer)": m_t,
        "ITT (JL − C)": itt,
        "SE (robust)": se,
        "Std. Diff (baseline)": sdiff
    })

if missing_cols:
    print("[warn] Missing columns skipped:", missing_cols)

tab2 = pd.DataFrame(rows)

# guard: if empty, print diagnostics and stop gracefully
if tab2.empty:
    print("[warn] No rows produced for outcomes table — nothing to write.")
else:
    # optional: order rows in a pleasing way without relying on 'Code'
    order_pref = [
        "Baseline consumption (log1p)",
        "Endline consumption (log1p)",
        "Baseline food exp. (log1p)",
        "Endline food exp. (log1p)",
        "Baseline business profit (levels)",
        "Endline business profit (levels)",
        "Baseline enterprise (0/1)",
        "Endline enterprise (0/1)",
    ]
    tab2["ord"] = tab2["Outcome"].apply(lambda x: order_pref.index(x) if x in order_pref else 999)
    tab2 = tab2.sort_values(["ord","Outcome"]).drop(columns=["ord"])

    # write CSV
    csv_path = os.path.join(TABLE_DIR, "table2_outcomes.csv")
    tab2.to_csv(csv_path, index=False)
    print(f"[ok] wrote {csv_path} (rows={len(tab2)})")

    # write LaTeX
    tex_path = os.path.join(TABLE_DIR, "table2_outcomes.tex")
    with open(tex_path, "w") as f:
        f.write("\\begin{table}[H]\\centering\n")
        f.write("\\caption{Outcomes: Baseline/Endline Means and ITT (cluster-robust)}\n")
        f.write("\\label{tab:outcomes}\n")
        f.write("\\begin{tabular}{lrrrrrrrr}\n\\toprule\n")
        f.write("Outcome & N & Mean (All) & SD (All) & Mean (Control) & Mean (JL Offer) & ITT (JL$-$C) & SE & Std. Diff (baseline) \\\\\n\\midrule\n")
        for _, r in tab2.iterrows():
            f.write(
                f"{r['Outcome']} & {r['N']:d} & "
                f"{r['Mean (All)']:.3f} & {r['SD (All)']:.3f} & "
                f"{r['Mean (Control)']:.3f} & {r['Mean (JL Offer)']:.3f} & "
                f"{(r['ITT (JL − C)'] if pd.notna(r['ITT (JL − C)']) else float('nan')):.3f} & "
                f"{(r['SE (robust)'] if pd.notna(r['SE (robust)']) else float('nan')):.3f} & "
                f"{(r['Std. Diff (baseline)'] if pd.notna(r['Std. Diff (baseline)']) else float('nan')):.3f} \\\\\n"
            )
        f.write("\\bottomrule\n\\end{tabular}\n")
        f.write("\\begin{minipage}{0.95\\linewidth}\\vspace{0.6ex}\\footnotesize\n")
        f.write("Notes: ITT estimated by OLS of outcome on a joint-liability offer indicator with village-clustered standard errors when a cluster id is available; otherwise HC2 robust SEs. ")
        f.write("Consumption and food outcomes are log1p-transformed; profits are in levels unless otherwise noted. ")
        f.write("Standardized differences are shown only for baseline outcomes as a balance diagnostic. ")
        f.write("\\end{minipage}\n\\end{table}\n")
    print(f"[ok] wrote {tex_path}")

# In LaTeX: \input{tables/table2_outcomes.tex}


# Statistics

In [ ]:
# ==========================================
# ANCOVA: y1 ~ y0 + Treatment + ALL baseline controls
# Showing Coefficients for All Control Variables
# Robust to dtype/object issues
# ==========================================
import numpy as np, pandas as pd, statsmodels.api as sm
import os
os.makedirs("tables", exist_ok=True)

# 0) Align unscaled feature matrix to Y rows
X_unscaled = X_proc.loc[Y_use.index].copy()   # pre-scale, imputed + one-hot

# 1) Column lists
drop_keys = {"treat_any","treat_indiv","treat_group","baselineY"}
baseline_controls = [c for c in X_unscaled.columns if c not in drop_keys]

cols_any = ["baselineY","treat_any"] + baseline_controls
cols_two = ["baselineY","treat_indiv","treat_group"] + baseline_controls

# 2) Helper: make design numeric and clean
def make_numeric_design(df, cols):
    X = df[cols].copy()

    # De-duplicate columns if any (keep first occurrence)
    if X.columns.duplicated().any():
        dup = X.columns[X.columns.duplicated()].tolist()
        print(f"[warn] dropping duplicate columns: {dup[:8]}{'...' if len(dup)>8 else ''}")
        X = X.loc[:, ~X.columns.duplicated()]

    # Cast booleans → int8
    bool_cols = [c for c in X.columns if X[c].dtype == bool]
    if bool_cols:
        X[bool_cols] = X[bool_cols].astype("int8")

    # Category → codes (missing becomes NaN, fill later)
    cat_cols = [c for c in X.columns if str(X[c].dtype) == "category"]
    for c in cat_cols:
        X[c] = X[c].cat.codes.replace(-1, np.nan)

    # Object → numeric (coerce)
    obj_cols = [c for c in X.columns if X[c].dtype == "O"]
    if obj_cols:
        print(f"[fix] coercing object columns to numeric: {obj_cols[:8]}{'...' if len(obj_cols)>8 else ''}")
        for c in obj_cols:
            X[c] = pd.to_numeric(X[c], errors="coerce")

    # Replace inf, then fill remaining NaN with column mean
    X = X.replace([np.inf, -np.inf], np.nan)
    for c in X.columns:
        if X[c].isna().any():
            X[c] = X[c].fillna(X[c].mean())

    # Final cast to float64
    X = X.astype("float64")

    # Sanity check: any non-numeric left?
    bad = [c for c in X.columns if not np.issubdtype(X[c].dtype, np.number)]
    assert not bad, f"Non-numeric columns remain: {bad}"
    return X

X_any = make_numeric_design(X_unscaled, cols_any)
X_two = make_numeric_design(X_unscaled, cols_two)

# 3) Cluster variable (village-level)
cluster_candidates = ["village_id","cluster_id","community_id","village","cluster","soum","aimag"]
cluster_col = next((c for c in cluster_candidates if c in Merged.columns), None)
clusters = Merged.loc[Y_use.index, cluster_col] if cluster_col else pd.Series(np.arange(len(Y_use)), index=Y_use.index)

# 4) Fit function with clustered SEs
def fit_ancova(y, X, clusters):
    y = np.asarray(y, dtype="float64")  # ensure numeric
    X = sm.add_constant(X, has_constant="add")
    model = sm.OLS(y, X, missing="drop")
    res = model.fit(cov_type="cluster", cov_kwds={"groups": clusters})
    return res

res_any = fit_ancova(Y_use, X_any, clusters)
res_two = fit_ancova(Y_use, X_two, clusters)

print("\n=== ANCOVA (Any offer) ===")
print(res_any.summary())

print("\n=== ANCOVA (Two arms: individual & joint liability) ===")
print(res_two.summary())

# 5) Save key coefficients for quick tables
pd.DataFrame({
    "coef": res_any.params[["treat_any","baselineY"]],
    "se_cluster": res_any.bse[["treat_any","baselineY"]],
    "pval": res_any.pvalues[["treat_any","baselineY"]],
}).to_csv("tables/ancova_any_offer_key.csv")

pd.DataFrame({
    "coef": res_two.params[["treat_indiv","treat_group","baselineY"]],
    "se_cluster": res_two.bse[["treat_indiv","treat_group","baselineY"]],
    "pval": res_two.pvalues[["treat_indiv","treat_group","baselineY"]],
}).to_csv("tables/ancova_two_arm_key.csv")

print("[write] tables/ancova_any_offer_key.csv, tables/ancova_two_arm_key.csv")


In [2]:
# ==========================================
# OLS (ANCOVA), report-only table: Constant, BaselineY, Treatment(s)
# ==========================================
import os, numpy as np, pandas as pd, statsmodels.api as sm

os.makedirs("tables", exist_ok=True)

# 0) Align to kept rows and use the unscaled, imputed, one-hot matrix
X_unscaled = X_proc.loc[Y_use.index].copy()

# 1) Build column sets
drop_keys = {"treat_any","treat_indiv","treat_group","baselineY"}
baseline_controls = [c for c in X_unscaled.columns if c not in drop_keys]

cols_any = ["baselineY","treat_any"] + baseline_controls
cols_two = ["baselineY","treat_indiv","treat_group"] + baseline_controls

# 2) Clean numeric design (handles object/booleans, fills NaN with mean)
def make_numeric_design(df, cols):
    X = df[cols].copy()
    # dedupe
    if X.columns.duplicated().any():
        X = X.loc[:, ~X.columns.duplicated()]
    # bool -> int8
    for c in X.columns:
        if X[c].dtype == bool:
            X[c] = X[c].astype("int8")
    # category -> codes
    for c in X.columns:
        if str(X[c].dtype) == "category":
            X[c] = X[c].cat.codes.replace(-1, np.nan)
    # object -> numeric
    obj_cols = [c for c in X.columns if X[c].dtype == "O"]
    for c in obj_cols:
        X[c] = pd.to_numeric(X[c], errors="coerce")
    # fill and cast
    X = X.replace([np.inf, -np.inf], np.nan)
    for c in X.columns:
        if X[c].isna().any():
            X[c] = X[c].fillna(X[c].mean())
    return X.astype("float64")

X_any = make_numeric_design(X_unscaled, cols_any)
X_two = make_numeric_design(X_unscaled, cols_two)

# 3) Cluster ids (village-level if available)
cluster_candidates = ["village_id","cluster_id","community_id","village","cluster","soum","aimag"]
cluster_col = next((c for c in cluster_candidates if c in Merged.columns), None)
clusters = (Merged.loc[Y_use.index, cluster_col]
            if cluster_col else pd.Series(np.arange(len(Y_use)), index=Y_use.index))

# 4) Fit with village-clustered SEs
def fit_clustered(y, X, clusters):
    y = np.asarray(y, dtype="float64")
    X = sm.add_constant(X, has_constant="add")
    model = sm.OLS(y, X, missing="drop")
    return model.fit(cov_type="cluster", cov_kwds={"groups": clusters})

res_any = fit_clustered(Y_use, X_any, clusters)
res_two = fit_clustered(Y_use, X_two, clusters)

# 5) Build a compact report table (only selected rows)
def stars(p):
    return "" if p>=0.1 else "*" if p>=0.05 else "**" if p>=0.01 else "***"

def pick_rows(res, wanted):
    b, se, p = res.params, res.bse, res.pvalues
    out = []
    for lab, key in wanted:
        if key not in b.index and key is not None:
            out.append((lab, ""))  # blank if not in this spec
        elif key is None:  # constant row uses 'const'
            val = b.get("const", np.nan); s = se.get("const", np.nan); pv = p.get("const", 1.0)
            out.append((lab, f"{val:0.3f}{stars(pv)}\n({s:0.3f})"))
        else:
            val = b[key]; s = se[key]; pv = p[key]
            out.append((lab, f"{val:0.3f}{stars(pv)}\n({s:0.3f})"))
    return out

# Rows to display (order)
rows_any = [("Treatment (Any offer)", "treat_any"),
            ("Baseline outcome",       "baselineY"),
            ("Constant",               None)]
rows_two = [("Treatment: Individual", "treat_indiv"),
            ("Treatment: Joint",      "treat_group"),
            ("Baseline outcome",      "baselineY"),
            ("Constant",              None)]

col_any  = pick_rows(res_any, rows_any)
col_two  = pick_rows(res_two, rows_two)

# 6) Assemble into a DataFrame for LaTeX
# Union the row labels in a sensible order
index = [r[0] for r in rows_two]  # superset order
report = pd.DataFrame(index=index, columns=["(1) Any offer","(2) Individual/Joint"])
# fill column (1)
for lab, val in col_any:
    if lab not in report.index:
        report.loc[lab] = ["", ""]
    report.loc[lab, "(1) Any offer"] = val
# fill column (2)
for lab, val in col_two:
    if lab not in report.index:
        report.loc[lab] = ["", ""]
    report.loc[lab, "(2) Individual/Joint"] = val

# Footer stats
footer = pd.DataFrame({
    "(1) Any offer":        [int(res_any.nobs),  res_any.rsquared],
    "(2) Individual/Joint": [int(res_two.nobs),  res_two.rsquared],
}, index=["Observations","R-squared"])

# 7) Minimal LaTeX (no threeparttable). Booktabs optional.
use_booktabs = True
def to_latex_min(tbl, foot, dep_label, controls_note, cluster_name):
    cols_spec = "l" + "c"*tbl.shape[1]
    lines = []
    lines.append("\\begin{table}[!htbp]\\centering")
    lines.append("\\caption{ANCOVA: Outcome on Baseline, Treatment, and All Baseline Controls}")
    lines.append("\\label{tab:ancova_main}")
    if use_booktabs:
        lines.append("\\begin{tabular}{"+cols_spec+"}")
        lines.append("\\toprule")
    else:
        lines.append("\\begin{tabular}{"+cols_spec+"}")
        lines.append("\\hline")
    lines.append(" & " + " & ".join(tbl.columns) + " \\\\")
    lines.append("\\midrule" if use_booktabs else "\\hline")
    for r in tbl.index:
        c1 = tbl.loc[r, tbl.columns[0]] if pd.notna(tbl.loc[r, tbl.columns[0]]) else ""
        c2 = tbl.loc[r, tbl.columns[1]] if pd.notna(tbl.loc[r, tbl.columns[1]]) else ""
        lines.append(f"{r} & {c1} & {c2} \\\\")
    lines.append("\\midrule" if use_booktabs else "\\hline")
    lines.append(f"Observations & {foot.loc['Observations','(1) Any offer']} & {foot.loc['Observations','(2) Individual/Joint']} \\\\")
    lines.append(f"R-squared & {foot.loc['R-squared','(1) Any offer']:.3f} & {foot.loc['R-squared','(2) Individual/Joint']:.3f} \\\\")
    lines.append("\\bottomrule" if use_booktabs else "\\hline")
    lines.append("\\end{tabular}")
    # Plain minipage note
    lines.append("")
    lines.append("\\par\\vspace{0.5ex}")
    lines.append("\\begin{minipage}{0.94\\linewidth}\\footnotesize")
    lines.append(f"Notes: Dependent variable {dep_label}. Heteroskedasticity- and cluster-robust standard errors in parentheses (clustered by {cluster_name}). ")
    lines.append("All specifications include the full set of baseline controls (unreported) and province/aimag fixed effects if present in the baseline set. ")
    lines.append("*, **, *** denote $p<0.05, p<0.01, p<0.001$, respectively.")
    lines.append("\\end{minipage}")
    lines.append("\\end{table}")
    return "\n".join(lines)

dep_label = f"$\\log(1+{Y_col})$"
cluster_name = cluster_col if cluster_col else "village"
latex_str = to_latex_min(report, footer, dep_label,
                         controls_note="All baseline controls included.",
                         cluster_name=cluster_name)

with open("tables/ancova_main.tex","w") as f:
    f.write(latex_str)

# Also save a CSV copy (handy for quick checks)
report.assign(**{"Obs (1)": int(res_any.nobs),
                 "Obs (2)": int(res_two.nobs)}).to_csv("tables/ancova_main.csv")

print("[write] tables/ancova_main.tex, tables/ancova_main.csv")


/opt/conda/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1884: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self.cov_params()))
/opt/conda/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1884: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self.cov_params()))


[write] tables/ancova_main.tex, tables/ancova_main.csv


In [6]:
# ==========================================
# Baseline OLS (ANCOVA) — simple & numeric-safe
# Shows only: Constant, baselineY, Treatment(s)
# Keeps all other baseline features as controls (unreported)
# ==========================================
import numpy as np, pandas as pd, statsmodels.api as sm, os
os.makedirs("tables", exist_ok=True)

assert {"X_proc","Y_use","Merged"}.issubset(globals()), "Run your cleaning cell first."

# ---- 0) Align rows
idx = X_proc.index.intersection(Y_use.index).intersection(Merged.index)
Y = Y_use.loc[idx].astype(float)
X_all = X_proc.loc[idx].copy()

# ---- 1) Build two specs (any-offer; two-arm)
focus_any = [c for c in ["baselineY","treat_any"] if c in X_all.columns]
focus_two = ["baselineY"] + [c for c in ["treat_indiv","treat_group"] if c in X_all.columns]

controls = X_all.columns.difference(set(focus_any + focus_two))  # all other baseline controls

X_any = pd.concat([X_all[focus_any], X_all[controls]], axis=1)
X_two = pd.concat([X_all[focus_two], X_all[controls]], axis=1)

# ---- 2) Ensure fully numeric design: get_dummies on any non-numeric, cast to float, fill NaN
def ensure_numeric_df(X: pd.DataFrame) -> pd.DataFrame:
    X = X.copy()
    # booleans -> ints
    for c in X.columns:
        if pd.api.types.is_bool_dtype(X[c]):
            X[c] = X[c].astype(np.int8)
    non_num = [c for c in X.columns if not pd.api.types.is_numeric_dtype(X[c])]
    if non_num:
        X = pd.concat([X.drop(columns=non_num),
                       pd.get_dummies(X[non_num], drop_first=True, dtype=float)], axis=1)
    # replace inf and fill NaN, cast to float
    X = X.replace([np.inf, -np.inf], np.nan)
    for c in X.columns:
        if X[c].isna().any():
            X[c] = X[c].astype(float).fillna(X[c].mean())
        else:
            X[c] = X[c].astype(float)
    # de-duplicate columns if any
    if X.columns.duplicated().any():
        X = X.loc[:, ~X.columns.duplicated()]
    return X

X_any = ensure_numeric_df(X_any)
X_two = ensure_numeric_df(X_two)

# ---- 3) Choose covariance: cluster by village if sensible, else HC1
cluster_candidates = ["village_id","village","cluster_id","community_id","soum","aimag"]
cluster_col = next((c for c in cluster_candidates if c in Merged.columns), None)
cov_type, cov_kw = "HC1", {}
if cluster_col is not None:
    g = Merged.loc[idx, cluster_col]
    if g.notna().all() and g.nunique() >= 20:
        cov_type, cov_kw = "cluster", {"groups": g.astype("object").values}
print(f"[cov] Using cov_type='{cov_type}'" + (f" clustered by {cluster_col}" if cov_type=='cluster' else " (HC1 robust)"))

# ---- 4) Fit helper
def fit_ancova(y: pd.Series, X: pd.DataFrame):
    X_use = sm.add_constant(X, has_constant="add")  # add intercept
    model = sm.OLS(y.values, X_use.values)
    res = model.fit(cov_type=cov_type, cov_kwds=cov_kw)
    res._xnames = ["const"] + list(X.columns)  # keep names for reporting
    return res

res_any = fit_ancova(Y, X_any)
res_two = fit_ancova(Y, X_two)

# ---- 5) Compact report: Constant, baselineY, and treatment(s)
def stars(p): return "***" if p<0.001 else "**" if p<0.01 else "*" if p<0.05 else ""
def extract(res, keys):
    b  = pd.Series(res.params, index=getattr(res, "_xnames"))
    se = pd.Series(res.bse,    index=getattr(res, "_xnames"))
    p  = pd.Series(res.pvalues,index=getattr(res, "_xnames"))
    out = []
    for lab, key in keys:
        nm = "const" if key is None else key
        val = b.get(nm, np.nan); s = se.get(nm, np.nan); pv = p.get(nm, 1.0)
        out.append(f"{val:.3f}{stars(pv)}\n({s:.3f})" if pd.notna(val) else "")
    return out

rows_any = [("Constant", None),
            ("Baseline outcome", "baselineY"),
            ("Treatment (Any offer)", "treat_any")]
rows_two = [("Constant", None),
            ("Baseline outcome", "baselineY"),
            ("Treatment: Individual", "treat_indiv"),
            ("Treatment: Joint", "treat_group")]

col_any = extract(res_any, rows_any)
col_two = extract(res_two, rows_two)

index = [r[0] for r in rows_two]
tbl = pd.DataFrame(index=index, columns=["(1) Any offer","(2) Individual/Joint"])
for (lab,_), val in zip(rows_any, col_any): tbl.loc[lab, "(1) Any offer"] = val
for (lab,_), val in zip(rows_two, col_two): tbl.loc[lab, "(2) Individual/Joint"] = val

footer = pd.DataFrame({
    "(1) Any offer":        [int(res_any.nobs),  res_any.rsquared],
    "(2) Individual/Joint": [int(res_two.nobs),  res_two.rsquared],
}, index=["Observations","R-squared"])

print("\n=== ANCOVA (reporting subset) ==="); print(tbl)
print("\n=== Footer ==="); print(footer)

# ---- 6) Minimal LaTeX export
use_booktabs = True
dep_label = f"$\\log(1+{Y.name})$" if isinstance(Y.name, str) else "$\\log(1+Y)$"
cluster_txt = f"clustered by {cluster_col}" if cov_type=="cluster" else "HC1 robust"

def latex_table(tbl, foot, caption="ANCOVA: Outcome on Baseline, Treatment, and Baseline Controls", label="tab:ancova_main"):
    cs = "l" + "c"*tbl.shape[1]
    L = []
    L += [r"\begin{table}[!htbp]\centering", f"\caption{{{caption}}}", f"\label{{{label}}}", r"\small",
          r"\begin{tabular}{"+cs+r"}", (r"\toprule" if use_booktabs else r"\hline"),
          " & " + " & ".join(tbl.columns) + r" \\",
          (r"\midrule" if use_booktabs else r"\hline")]
    for r in tbl.index:
        L.append(f"{r} & {tbl.loc[r, tbl.columns[0]] or ''} & {tbl.loc[r, tbl.columns[1]] or ''} \\\\")
    L += [(r"\midrule" if use_booktabs else r"\hline"),
          f"Observations & {foot.loc['Observations','(1) Any offer']} & {foot.loc['Observations','(2) Individual/Joint']} \\\\",
          f"R-squared & {foot.loc['R-squared','(1) Any offer']:.3f} & {foot.loc['R-squared','(2) Individual/Joint']:.3f} \\\\",
          (r"\bottomrule" if use_booktabs else r"\hline"),
          r"\end{tabular}", "", r"\vspace{0.5ex}",
          r"\begin{minipage}{0.94\linewidth}\footnotesize",
          f"Notes: Dependent variable {dep_label}. Standard errors are {cluster_txt}. ",
          r"All specifications include the full set of baseline controls (unreported). ",
          r"*, **, *** denote $p<0.05, p<0.01, p<0.001$, respectively.",
          r"\end{minipage}", r"\end{table}"]
    return "\n".join(L)

with open("tables/ancova_main.tex","w") as f:
    f.write(latex_table(tbl, footer))
tbl.assign(**{"Obs (1)": int(res_any.nobs), "Obs (2)": int(res_two.nobs)}).to_csv("tables/ancova_main.csv")
print("\n[write] tables/ancova_main.tex, tables/ancova_main.csv")


<>:117: SyntaxWarning: invalid escape sequence '\c'
<>:117: SyntaxWarning: invalid escape sequence '\l'
<>:117: SyntaxWarning: invalid escape sequence '\c'
<>:117: SyntaxWarning: invalid escape sequence '\l'
/tmp/ipykernel_1218/1348326420.py:117: SyntaxWarning: invalid escape sequence '\c'
  L += [r"\begin{table}[!htbp]\centering", f"\caption{{{caption}}}", f"\label{{{label}}}", r"\small",
/tmp/ipykernel_1218/1348326420.py:117: SyntaxWarning: invalid escape sequence '\l'
  L += [r"\begin{table}[!htbp]\centering", f"\caption{{{caption}}}", f"\label{{{label}}}", r"\small",


[cov] Using cov_type='HC1' (HC1 robust)

=== ANCOVA (reporting subset) ===
                          (1) Any offer (2) Individual/Joint
Constant               18.761\n(19.744)     18.600\n(19.793)
Baseline outcome         0.336\n(0.180)       0.331\n(0.181)
Treatment: Individual               NaN       0.012\n(0.063)
Treatment: Joint                    NaN      -0.034\n(0.068)
Treatment (Any offer)   -0.005\n(0.059)                  NaN

=== Footer ===
              (1) Any offer  (2) Individual/Joint
Observations       961.0000            961.000000
R-squared            0.9429              0.943014

[write] tables/ancova_main.tex, tables/ancova_main.csv


In [ ]:
# ============================================================
# Post-LASSO ANCOVA: y1 ~ y0 + Treatment + (Top-20 LASSO controls only)
# ============================================================
import os, numpy as np, pandas as pd, statsmodels.api as sm, difflib

os.makedirs("tables", exist_ok=True)

# 0) Align design rows to the analysis sample
X_unscaled = X_proc.loc[Y_use.index].copy()

# 1) Top-20 LASSO variables as shown in your figure legend
#    (edit here if your labels differ)
LASSO_TOP20_RAW = [
    "redmeat2.x","butter_ps0","transer7.x","transer8.x","vehicl11.x",
    "schoo12.x","cladul12.x","schoo11.x","othecom7.x","butter1.x",
    "baselineY","milk2.x","othecom8.x","flour2.x","vegetab1.x","rice1.x",
    "othdair2.x","choco3.x","othdair1.x","recreat7.x"
]

# 2) Clean the list: remove 'baselineY' (we add it explicitly), fuzzy-match near-misses
ALL_COLS = X_unscaled.columns.tolist()
want = [v for v in LASSO_TOP20_RAW if v != "baselineY"]

present = [v for v in want if v in ALL_COLS]
missing = [v for v in want if v not in ALL_COLS]

# Try gentle fuzzy matches for missing labels (e.g., tiny typos)
matched = {}
for v in missing:
    cand = difflib.get_close_matches(v, ALL_COLS, n=1, cutoff=0.8)
    if cand:
        matched[v] = cand[0]
        present.append(cand[0])

if missing:
    still_missing = [v for v in missing if v not in matched]
    if still_missing:
        print(f"[postLASSO] WARNING: not found (no close match): {still_missing}")
if matched:
    print("[postLASSO] Fuzzy-matched controls:", matched)

postlasso_controls = sorted(set(present))  # unique, sorted for reproducibility
print(f"[postLASSO] Using {len(postlasso_controls)} LASSO-selected controls.")

# 3) Build column sets for pooled any-offer and two-arm specs
cols_any = ["baselineY","treat_any"] + postlasso_controls
cols_two = ["baselineY","treat_indiv","treat_group"] + postlasso_controls

# 4) Make fully numeric design (handles object/booleans; fills NaN with means)
def make_numeric_design(df, cols):
    X = df[cols].copy()
    # De-duplicate columns
    if X.columns.duplicated().any():
        X = X.loc[:, ~X.columns.duplicated()]
    # Cast types
    for c in X.columns:
        if X[c].dtype == bool:
            X[c] = X[c].astype("int8")
        elif str(X[c].dtype) == "category":
            X[c] = X[c].cat.codes.replace(-1, np.nan)
        elif X[c].dtype == "O":
            X[c] = pd.to_numeric(X[c], errors="coerce")
    # Clean infinities, fill NAs with means, cast float64
    X = X.replace([np.inf, -np.inf], np.nan)
    for c in X.columns:
        if X[c].isna().any():
            X[c] = X[c].fillna(X[c].mean())
    return X.astype("float64")

X_any = make_numeric_design(X_unscaled, cols_any)
X_two = make_numeric_design(X_unscaled, cols_two)

# 5) Cluster ids (village-level if available)
cluster_candidates = ["village_id","cluster_id","community_id","village","cluster","soum","aimag"]
cluster_col = next((c for c in cluster_candidates if c in Merged.columns), None)
clusters = (Merged.loc[Y_use.index, cluster_col]
            if cluster_col else pd.Series(np.arange(len(Y_use)), index=Y_use.index))

# 6) Fit OLS with village-clustered SEs
def fit_clustered(y, X, clusters):
    y = np.asarray(y, dtype="float64")
    X = sm.add_constant(X, has_constant="add")
    model = sm.OLS(y, X, missing="drop")
    return model.fit(cov_type="cluster", cov_kwds={"groups": clusters})

res_any = fit_clustered(Y_use, X_any, clusters)
res_two = fit_clustered(Y_use, X_two, clusters)

print("\n=== Post-LASSO ANCOVA (Any offer) ===")
print(res_any.summary())
print("\n=== Post-LASSO ANCOVA (Two arms) ===")
print(res_two.summary())

# 7) Build a compact report table (only Treatment(s), BaselineY, Constant)
def stars(p):
    return "" if p>=0.1 else "*" if p>=0.05 else "**" if p>=0.01 else "***"

def pick_rows(res, wanted):
    b, se, p = res.params, res.bse, res.pvalues
    out = []
    for lab, key in wanted:
        if key is None:
            val = b.get("const", np.nan); s = se.get("const", np.nan); pv = p.get("const", 1.0)
        else:
            val = b[key]; s = se[key]; pv = p[key]
        out.append((lab, f"{val:0.3f}{stars(pv)}\n({s:0.3f})"))
    return out

rows_any = [("Treatment (Any offer)","treat_any"),
            ("Baseline outcome","baselineY"),
            ("Constant",None)]
rows_two = [("Treatment: Individual","treat_indiv"),
            ("Treatment: Joint","treat_group"),
            ("Baseline outcome","baselineY"),
            ("Constant",None)]

col_any = pick_rows(res_any, rows_any)
col_two = pick_rows(res_two, rows_two)

index = [r[0] for r in rows_two]
report = pd.DataFrame(index=index, columns=["(1) Any offer","(2) Individual/Joint"])
for lab, val in col_any:
    report.loc[lab, "(1) Any offer"] = val
for lab, val in col_two:
    report.loc[lab, "(2) Individual/Joint"] = val

footer = pd.DataFrame({
    "(1) Any offer":[int(res_any.nobs), res_any.rsquared],
    "(2) Individual/Joint":[int(res_two.nobs), res_two.rsquared],
}, index=["Observations","R-squared"])

# 8) Emit LaTeX (post-LASSO caption + note)
use_booktabs = True
def to_latex_min(tbl, foot, dep_label, cluster_name, k_controls):
    cols_spec = "l" + "c"*tbl.shape[1]
    L = []
    L.append("\\begin{table}[!htbp]\\centering")
    L.append("\\caption{Post-LASSO ANCOVA: Outcome on Baseline, Treatment, and Selected Controls}")
    L.append("\\label{tab:ancova_postlasso}")
    L.append("\\begin{tabular}{"+cols_spec+"}")
    L.append("\\toprule" if use_booktabs else "\\hline")
    L.append(" & " + " & ".join(tbl.columns) + " \\\\")
    L.append("\\midrule" if use_booktabs else "\\hline")
    for r in tbl.index:
        L.append(f"{r} & {tbl.loc[r, tbl.columns[0]] or ''} & {tbl.loc[r, tbl.columns[1]] or ''} \\\\")
    L.append("\\midrule" if use_booktabs else "\\hline")
    L.append(f"Observations & {foot.loc['Observations','(1) Any offer']} & {foot.loc['Observations','(2) Individual/Joint']} \\\\")
    L.append(f"R-squared & {foot.loc['R-squared','(1) Any offer']:.3f} & {foot.loc['R-squared','(2) Individual/Joint']:.3f} \\\\")
    L.append("\\bottomrule" if use_booktabs else "\\hline")
    L.append("\\end{tabular}")
    L.append("\\par\\vspace{0.5ex}")
    L.append("\\begin{minipage}{0.94\\linewidth}\\footnotesize")
    L.append(f"Notes: Dependent variable {dep_label}. Cluster-robust SEs in parentheses (clustered by {cluster_name}). ")
    L.append(f"Controls are the LASSO-selected top {k_controls} baseline covariates from Appendix Figure~\\ref{{fig:lasso-top20}}, rescaled to $[0,1]$. ")
    L.append("Province/stratum indicators are omitted in this sparse specification by design. ")
    L.append("*, **, *** denote $p<0.05, p<0.01, p<0.001$.")
    L.append("\\end{minipage}")
    L.append("\\end{table}")
    return "\n".join(L)

dep_label = f"$\\log(1+{Y_col})$"
cluster_name = cluster_col if cluster_col else "village"
latex_str = to_latex_min(report, footer, dep_label, cluster_name, len(postlasso_controls))

with open("tables/ancova_postlasso.tex","w") as f:
    f.write(latex_str)

# 9) Save audit trail of which controls were used
pd.Series(postlasso_controls, name="postLASSO_controls").to_csv("tables/postlasso_controls.csv", index=False)

print("[write] tables/ancova_postlasso.tex, tables/postlasso_controls.csv")


In [ ]:
# ============================================================
# Post-LASSO ANCOVA with province/stratum FE + "hot" baselines
# y1 ~ y0 + Treatment + (Top-20 LASSO) + FE(stratum) + Always-keep
# ============================================================
import os, re, difflib
import numpy as np, pandas as pd, statsmodels.api as sm

os.makedirs("tables", exist_ok=True)

# 0) Align design rows to the analysis sample
X_unscaled = X_proc.loc[Y_use.index].copy()   # your imputed, one-hot baseline matrix

# 1) LASSO top-20 (from your figure legend)
LASSO_TOP20_RAW = [
    "redmeat2.x","butter_ps0","transer7.x","transer8.x","vehicl11.x",
    "schoo12.x","cladul12.x","schoo11.x","othecom7.x","butter1.x",
    "baselineY","milk2.x","othecom8.x","flour2.x","vegetab1.x","rice1.x",
    "othdair2.x","choco3.x","othdair1.x","recreat7.x"
]

# 2) Build: LASSO-selected controls (minus baselineY), add FE + Always-keep
ALL_COLS = X_unscaled.columns.tolist()
want = [v for v in LASSO_TOP20_RAW if v != "baselineY"]

# exact matches
present = [v for v in want if v in ALL_COLS]
missing = [v for v in want if v not in ALL_COLS]

# gentle fuzzy matches for near-misses (typos, suffixes)
matched = {}
for v in missing:
    cand = difflib.get_close_matches(v, ALL_COLS, n=1, cutoff=0.8)
    if cand:
        matched[v] = cand[0]
        present.append(cand[0])

if matched:
    print("[postLASSO] Fuzzy-matched:", matched)
if missing:
    still_missing = [v for v in missing if v not in matched]
    if still_missing:
        print(f"[postLASSO] WARNING not found: {still_missing}")

lasso_controls = set(present)

# province/stratum fixed effects already in your design? grab any aimag/soum/region/province dummies
FE_KEYS = ("aimag", "province", "soum", "region")
fe_cols = [c for c in ALL_COLS if any(k in c.lower() for k in FE_KEYS)]

# Optional: keep only dummy-like FE columns (few unique values)
fe_cols = [c for c in fe_cols if X_unscaled[c].nunique(dropna=True) <= 50]
print(f"[postLASSO] Added FE columns: {len(fe_cols)}")

# "Always-keep" hot baseline controls (if present in your matrix)
ALWAYS_KEEP_SEEDS = [
    "age","age_head","hhsize","children","dependents","urban","married","educ_head",
    "enterprise","hours_ent","hours_wage","assets","profit"
]
def find_seed_cols(seeds, cols):
    picks = []
    for c in cols:
        name = c.lower()
        if any(re.search(rf"\b{re.escape(s)}(\b|[_.=])", name) for s in seeds):
            picks.append(c)
    return picks

hot_cols = find_seed_cols(ALWAYS_KEEP_SEEDS, ALL_COLS)
print(f"[postLASSO] Added 'hot' baseline controls: {len(hot_cols)}")

# final selected baseline controls
selected_controls = sorted(set(list(lasso_controls) + fe_cols + hot_cols))
print(f"[postLASSO] Total selected baseline controls: {len(selected_controls)}")

# 3) Column sets for pooled any-offer and two-arm specs
cols_any = ["baselineY","treat_any"] + selected_controls
cols_two = ["baselineY","treat_indiv","treat_group"] + selected_controls

# 4) Ensure numeric design
def make_numeric_design(df, cols):
    X = df[cols].copy()
    if X.columns.duplicated().any():
        X = X.loc[:, ~X.columns.duplicated()]
    for c in X.columns:
        if X[c].dtype == bool:
            X[c] = X[c].astype("int8")
        elif str(X[c].dtype) == "category":
            X[c] = X[c].cat.codes.replace(-1, np.nan)
        elif X[c].dtype == "O":
            X[c] = pd.to_numeric(X[c], errors="coerce")
    X = X.replace([np.inf, -np.inf], np.nan)
    for c in X.columns:
        if X[c].isna().any():
            X[c] = X[c].fillna(X[c].mean())
    return X.astype("float64")

X_any = make_numeric_design(X_unscaled, cols_any)
X_two = make_numeric_design(X_unscaled, cols_two)

# 5) Cluster ids (village-level if available)
cluster_candidates = ["village_id","cluster_id","community_id","village","cluster","soum","aimag"]
cluster_col = next((c for c in cluster_candidates if c in Merged.columns), None)
clusters = (Merged.loc[Y_use.index, cluster_col]
            if cluster_col else pd.Series(np.arange(len(Y_use)), index=Y_use.index))

# 6) Fit OLS with village-clustered SEs
def fit_clustered(y, X, clusters):
    y = np.asarray(y, dtype="float64")
    X = sm.add_constant(X, has_constant="add")
    model = sm.OLS(y, X, missing="drop")
    return model.fit(cov_type="cluster", cov_kwds={"groups": clusters})

res_any = fit_clustered(Y_use, X_any, clusters)
res_two = fit_clustered(Y_use, X_two, clusters)

print("\n=== Post-LASSO+FE ANCOVA (Any offer) ===")
print(res_any.summary())
print("\n=== Post-LASSO+FE ANCOVA (Two arms) ===")
print(res_two.summary())

# 7) Compact report (Treatment(s), BaselineY, Constant only)
def stars(p): return "" if p>=0.1 else "*" if p>=0.05 else "**" if p>=0.01 else "***"
def pick_rows(res, wanted):
    b, se, p = res.params, res.bse, res.pvalues
    out = []
    for lab, key in wanted:
        if key is None:
            val = b.get("const", np.nan); s = se.get("const", np.nan); pv = p.get("const", 1.0)
        else:
            val = b[key]; s = se[key]; pv = p[key]
        out.append((lab, f"{val:0.3f}{stars(pv)}\n({s:0.3f})"))
    return out

rows_any = [("Treatment (Any offer)","treat_any"),
            ("Baseline outcome","baselineY"),
            ("Constant",None)]
rows_two = [("Treatment: Individual","treat_indiv"),
            ("Treatment: Joint","treat_group"),
            ("Baseline outcome","baselineY"),
            ("Constant",None)]

col_any = pick_rows(res_any, rows_any)
col_two = pick_rows(res_two, rows_two)

index = [r[0] for r in rows_two]
report = pd.DataFrame(index=index, columns=["(1) Any offer","(2) Individual/Joint"])
for lab, val in col_any: report.loc[lab, "(1) Any offer"] = val
for lab, val in col_two: report.loc[lab, "(2) Individual/Joint"] = val

footer = pd.DataFrame({
    "(1) Any offer":[int(res_any.nobs), res_any.rsquared],
    "(2) Individual/Joint":[int(res_two.nobs), res_two.rsquared],
}, index=["Observations","R-squared"])

# 8) LaTeX table (note now says FE + hot controls included)
use_booktabs = True
def to_latex_min(tbl, foot, dep_label, cluster_name, k_lasso, k_fe, k_hot):
    spec = "l" + "c"*tbl.shape[1]
    L = []
    L.append("\\begin{table}[!htbp]\\centering")
    L.append("\\caption{Post-LASSO ANCOVA with Province/Stratum Fixed Effects and Hot Baseline Controls}")
    L.append("\\label{tab:ancova_postlasso}")
    L.append("\\begin{tabular}{"+spec+"}")
    L.append("\\toprule" if use_booktabs else "\\hline")
    L.append(" & " + " & ".join(tbl.columns) + " \\\\")
    L.append("\\midrule" if use_booktabs else "\\hline")
    for r in tbl.index:
        L.append(f"{r} & {tbl.loc[r, tbl.columns[0]] or ''} & {tbl.loc[r, tbl.columns[1]] or ''} \\\\")
    L.append("\\midrule" if use_booktabs else "\\hline")
    L.append(f"Observations & {foot.loc['Observations','(1) Any offer']} & {foot.loc['Observations','(2) Individual/Joint']} \\\\")
    L.append(f"R-squared & {foot.loc['R-squared','(1) Any offer']:.3f} & {foot.loc['R-squared','(2) Individual/Joint']:.3f} \\\\")
    L.append("\\bottomrule" if use_booktabs else "\\hline")
    L.append("\\end{tabular}")
    L.append("\\par\\vspace{0.5ex}")
    L.append("\\begin{minipage}{0.96\\linewidth}\\footnotesize")
    L.append(f"Notes: Dependent variable {dep_label}. Cluster-robust SEs in parentheses (clustered by {cluster_name}). ")
    L.append(f"Controls include: LASSO-selected top {k_lasso} covariates (Appendix Figure~\\ref{{fig:lasso-top20}}), "
             f"province/stratum fixed effects (\\(K={k_fe}\\)), and a small always-keep set of baseline demographics/enterprise proxies (\\(K={k_hot}\\)). ")
    L.append("All covariates are from the pre-treatment baseline; treatment coefficients retain the ITT interpretation.")
    L.append("\\end{minipage}")
    L.append("\\end{table}")
    return "\n".join(L)

dep_label = f"$\\log(1+{Y_col})$"
cluster_name = cluster_col if cluster_col else "village"
latex_str = to_latex_min(
    report, footer, dep_label, cluster_name,
    k_lasso=len(lasso_controls), k_fe=len(fe_cols), k_hot=len(hot_cols)
)

with open("tables/ancova_postlasso.tex","w") as f:
    f.write(latex_str)

# 9) Save audit trail (tidy long form + optional one-row summary)

# Long form: one row per variable with its type
audit_rows = (
    [{"type": "lasso", "name": c} for c in sorted(lasso_controls)] +
    [{"type": "fe",    "name": c} for c in sorted(fe_cols)] +
    [{"type": "hot",   "name": c} for c in sorted(set(hot_cols))]
)
audit_df = pd.DataFrame(audit_rows)
audit_df.to_csv("tables/postlasso_controls_audit.csv", index=False)

# Optional: compact one-row summary (handy for quick reading)
summary_df = pd.DataFrame({
    "k_lasso": [len(lasso_controls)],
    "k_fe":    [len(fe_cols)],
    "k_hot":   [len(set(hot_cols))],
    "lasso_cols": ["; ".join(sorted(lasso_controls))],
    "fe_cols":    ["; ".join(sorted(fe_cols))],
    "hot_cols":   ["; ".join(sorted(set(hot_cols)))]
})
summary_df.to_csv("tables/postlasso_controls_summary.csv", index=False)

print("[write] tables/postlasso_controls_audit.csv, tables/postlasso_controls_summary.csv")



In [ ]:
import re
import numpy as np
import pandas as pd

# 1) Heuristic search for endline take-up variables
#    Look for words: loan, borrow, credit, micro, MFI, take, adopt, client, member, etc.
takeup_pattern = re.compile(r"(adopt|take|took|borrow|loan|credit|micro|mfi|client|member|join)", re.I)

# Columns that *look* like endline (avoid baseline .x/0/base)
def looks_endline(name: str) -> bool:
    name = name.lower()
    return (name.endswith(".y") or
            re.search(r"(end|follow|post|1)\b", name)) and not (
            name.endswith(".x") or re.search(r"(0|base|baseline)\b", name))

cand_all = [c for c in Merged.columns if takeup_pattern.search(str(c))]
cand_end = [c for c in cand_all if looks_endline(str(c))]

# 2) Keep indicator-like variables
def is_indicator_like(s: pd.Series) -> bool:
    vals = pd.to_numeric(s, errors="coerce")
    u = set(vals.dropna().unique())
    return len(u) > 0 and u.issubset({0,1})

cand_bin = [c for c in cand_end if is_indicator_like(Merged[c])]

print("[adoption] candidates (binary, endline-like):", cand_bin[:25])

# 3) First-stage sanity check: adoption should be higher in treatment villages
df = Merged.loc[Y_use.index].copy()  # align to analysis sample
first_stage = []
for c in cand_bin:
    v = pd.to_numeric(df[c], errors="coerce")
    m_c = v[treat_any.loc[df.index]==0].mean()
    m_t = v[treat_any.loc[df.index]==1].mean()
    n   = v.notna().sum()
    first_stage.append((c, n, m_c, m_t, m_t-m_c))

first_stage = pd.DataFrame(first_stage, columns=["var","N","ctrl_mean","treat_mean","diff"])
first_stage = first_stage.sort_values("diff", ascending=False)
print(first_stage.head(15).to_string(index=False))

# 4) Pick your adoption D:
#    The best candidate is: (a) clearly binary, (b) measured at endline,
#    (c) has a sizable treat-control difference, and ideally (d) refers to the *partner MFI* specifically.
#    Suppose the top row looks good:
if not first_stage.empty:
    D_var = first_stage.iloc[0]["var"]
    print(f"[adoption] Suggested D variable: {D_var!r}")
else:
    D_var = None
